In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
df=pd.read_csv('df_all_d_new.csv',index_col=[0])

In [3]:
df_y=pd.read_csv('players_22.csv')
df_y=df_y[['sofifa_id','value_eur']]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (25,108) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df_y

,sofifa_id,value_eur
0,158023,78000000.0
1,188545,119500000.0
2,20801,45000000.0
3,190871,129000000.0
4,192985,125500000.0
...,...,...
19234,261962,70000.0
19235,262040,110000.0
19236,262760,100000.0
19237,262820,110000.0


In [5]:
df

,sofifa_id,physic_21,defending_21,dribbling_21,passing_21,shooting_21,pace_21,release_clause_eur_21,international_reputation_21,skill_moves_21,weak_foot_21,nation_team_id_21,club_contract_valid_until_21,league_level_21,club_team_id_21,weight_kg_21,height_cm_21,age_21,wage_eur_21,value_eur_21,potential_21,overall_21,physic_20,defending_20,dribbling_20,passing_20,shooting_20,pace_20,release_clause_eur_20,international_reputation_20,skill_moves_20,weak_foot_20,nation_team_id_20,club_contract_valid_until_20,league_level_20,club_team_id_20,weight_kg_20,height_cm_20,age_20,wage_eur_20,value_eur_20,potential_20,overall_20,physic_19,defending_19,dribbling_19,passing_19,shooting_19,pace_19,release_clause_eur_19,international_reputation_19,skill_moves_19,weak_foot_19,nation_team_id_19,club_contract_valid_until_19,league_level_19,club_team_id_19,weight_kg_19,height_cm_19,age_19,wage_eur_19,value_eur_19,potential_19,overall_19
0,158023,65.0,38.0,95.0,91.0,92.0,85.0,138400000.0,5.0,4.0,4.0,1369.0,2021.0,1.0,241.0,72.0,170.0,33.0,560000.0,103500000.0,93.0,93.0,66.0,39.0,96.0,92.0,92.0,87.0,195800000.0,5.0,4.0,4.0,NaN,2021.0,1.0,241.0,72.0,170.0,32.0,560000.0,95500000.0,94.0,94.0,61.0,32.0,96.0,88.0,91.0,88.0,226500000.0,5.0,4.0,4.0,1369.0,2021.0,1.0,241.0,72.0,170.0,31.0,575000.0,110500000.0,94.0,94.0
1,188545,82.0,43.0,85.0,78.0,91.0,78.0,132000000.0,4.0,4.0,4.0,NaN,2023.0,1.0,21.0,80.0,184.0,31.0,240000.0,111000000.0,91.0,91.0,82.0,41.0,85.0,74.0,87.0,77.0,106400000.0,4.0,4.0,4.0,1353.0,2021.0,1.0,21.0,80.0,184.0,30.0,230000.0,64500000.0,89.0,89.0,82.0,41.0,85.0,75.0,89.0,78.0,127100000.0,4.0,4.0,4.0,1353.0,2021.0,1.0,21.0,80.0,184.0,29.0,200000.0,77000000.0,90.0,90.0
2,20801,77.0,35.0,89.0,81.0,93.0,89.0,75900000.0,5.0,5.0,4.0,1354.0,2022.0,1.0,45.0,83.0,187.0,35.0,220000.0,63000000.0,92.0,92.0,78.0,35.0,89.0,82.0,93.0,90.0,96500000.0,5.0,5.0,4.0,1354.0,2022.0,1.0,45.0,83.0,187.0,34.0,410000.0,58500000.0,93.0,93.0,79.0,35.0,89.0,81.0,93.0,90.0,127100000.0,5.0,5.0,4.0,1354.0,2022.0,1.0,45.0,83.0,187.0,33.0,400000.0,77000000.0,94.0,94.0
3,190871,59.0,36.0,94.0,86.0,85.0,91.0,166500000.0,5.0,5.0,5.0,NaN,2022.0,1.0,73.0,68.0,175.0,28.0,270000.0,132000000.0,91.0,91.0,58.0,32.0,95.0,87.0,85.0,91.0,195200000.0,5.0,5.0,5.0,1370.0,2022.0,1.0,73.0,68.0,175.0,27.0,290000.0,105500000.0,92.0,92.0,59.0,32.0,95.0,83.0,84.0,92.0,228100000.0,5.0,5.0,5.0,1370.0,2022.0,1.0,73.0,68.0,175.0,26.0,300000.0,118500000.0,93.0,92.0
4,192985,78.0,64.0,88.0,93.0,86.0,76.0,161000000.0,4.0,4.0,5.0,1325.0,2023.0,1.0,10.0,70.0,181.0,29.0,370000.0,129000000.0,91.0,91.0,78.0,61.0,86.0,92.0,86.0,76.0,166500000.0,4.0,4.0,5.0,1325.0,2023.0,1.0,10.0,70.0,181.0,28.0,370000.0,90000000.0,91.0,91.0,78.0,60.0,87.0,92.0,86.0,77.0,196400000.0,4.0,4.0,5.0,1325.0,2023.0,1.0,10.0,70.0,181.0,27.0,350000.0,102000000.0,92.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45624,225340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45625,221338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45626,224829,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45627,220549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [6]:
df_merged=df.merge(df_y,on='sofifa_id',how='inner')
df_merged=df_merged.fillna(0)
df_merged=df_merged[:15000]
df_merged=df_merged.drop(columns='sofifa_id')

In [7]:
df_merged

,physic_21,defending_21,dribbling_21,passing_21,shooting_21,pace_21,release_clause_eur_21,international_reputation_21,skill_moves_21,weak_foot_21,nation_team_id_21,club_contract_valid_until_21,league_level_21,club_team_id_21,weight_kg_21,height_cm_21,age_21,wage_eur_21,value_eur_21,potential_21,overall_21,physic_20,defending_20,dribbling_20,passing_20,shooting_20,pace_20,release_clause_eur_20,international_reputation_20,skill_moves_20,weak_foot_20,nation_team_id_20,club_contract_valid_until_20,league_level_20,club_team_id_20,weight_kg_20,height_cm_20,age_20,wage_eur_20,value_eur_20,potential_20,overall_20,physic_19,defending_19,dribbling_19,passing_19,shooting_19,pace_19,release_clause_eur_19,international_reputation_19,skill_moves_19,weak_foot_19,nation_team_id_19,club_contract_valid_until_19,league_level_19,club_team_id_19,weight_kg_19,height_cm_19,age_19,wage_eur_19,value_eur_19,potential_19,overall_19,value_eur
0,65.0,38.0,95.0,91.0,92.0,85.0,138400000.0,5.0,4.0,4.0,1369.0,2021.0,1.0,241.0,72.0,170.0,33.0,560000.0,103500000.0,93.0,93.0,66.0,39.0,96.0,92.0,92.0,87.0,195800000.0,5.0,4.0,4.0,0.0,2021.0,1.0,241.0,72.0,170.0,32.0,560000.0,95500000.0,94.0,94.0,61.0,32.0,96.0,88.0,91.0,88.0,226500000.0,5.0,4.0,4.0,1369.0,2021.0,1.0,241.0,72.0,170.0,31.0,575000.0,110500000.0,94.0,94.0,78000000.0
1,82.0,43.0,85.0,78.0,91.0,78.0,132000000.0,4.0,4.0,4.0,0.0,2023.0,1.0,21.0,80.0,184.0,31.0,240000.0,111000000.0,91.0,91.0,82.0,41.0,85.0,74.0,87.0,77.0,106400000.0,4.0,4.0,4.0,1353.0,2021.0,1.0,21.0,80.0,184.0,30.0,230000.0,64500000.0,89.0,89.0,82.0,41.0,85.0,75.0,89.0,78.0,127100000.0,4.0,4.0,4.0,1353.0,2021.0,1.0,21.0,80.0,184.0,29.0,200000.0,77000000.0,90.0,90.0,119500000.0
2,77.0,35.0,89.0,81.0,93.0,89.0,75900000.0,5.0,5.0,4.0,1354.0,2022.0,1.0,45.0,83.0,187.0,35.0,220000.0,63000000.0,92.0,92.0,78.0,35.0,89.0,82.0,93.0,90.0,96500000.0,5.0,5.0,4.0,1354.0,2022.0,1.0,45.0,83.0,187.0,34.0,410000.0,58500000.0,93.0,93.0,79.0,35.0,89.0,81.0,93.0,90.0,127100000.0,5.0,5.0,4.0,1354.0,2022.0,1.0,45.0,83.0,187.0,33.0,400000.0,77000000.0,94.0,94.0,45000000.0
3,59.0,36.0,94.0,86.0,85.0,91.0,166500000.0,5.0,5.0,5.0,0.0,2022.0,1.0,73.0,68.0,175.0,28.0,270000.0,132000000.0,91.0,91.0,58.0,32.0,95.0,87.0,85.0,91.0,195200000.0,5.0,5.0,5.0,1370.0,2022.0,1.0,73.0,68.0,175.0,27.0,290000.0,105500000.0,92.0,92.0,59.0,32.0,95.0,83.0,84.0,92.0,228100000.0,5.0,5.0,5.0,1370.0,2022.0,1.0,73.0,68.0,175.0,26.0,300000.0,118500000.0,93.0,92.0,129000000.0
4,78.0,64.0,88.0,93.0,86.0,76.0,161000000.0,4.0,4.0,5.0,1325.0,2023.0,1.0,10.0,70.0,181.0,29.0,370000.0,129000000.0,91.0,91.0,78.0,61.0,86.0,92.0,86.0,76.0,166500000.0,4.0,4.0,5.0,1325.0,2023.0,1.0,10.0,70.0,181.0,28.0,370000.0,90000000.0,91.0,91.0,78.0,60.0,87.0,92.0,86.0,77.0,196400000.0,4.0,4.0,5.0,1325.0,2023.0,1.0,10.0,70.0,181.0,27.0,350000.0,102000000.0,92.0,91.0,125500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,425000.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90000.0
14997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,650000.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [8]:
from sklearn.preprocessing import StandardScaler
std=StandardScaler()
df_merged_fit=std.fit_transform(df_merged)
df_merged_fit=pd.DataFrame(df_merged_fit)
df_merged_fit

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
0,0.557208,-0.022299,1.603904,1.714619,1.947839,1.099224,12.285539,7.028481,1.667417,1.194068,-0.019214,0.511873,-0.069174,-0.650032,0.385467,0.347672,1.146081,25.223182,11.719789,1.171262,1.377849,0.719801,0.170122,1.710103,1.825753,2.029412,1.251877,16.420935,6.631331,1.719506,1.273306,-0.120368,0.651555,0.083083,-0.561749,0.531606,0.501037,1.180633,23.732108,15.295428,1.239160,1.437093,0.711703,0.076092,1.829643,1.815545,2.119192,1.389079,19.304116,6.431570,1.844064,1.382848,-0.011707,0.787347,0.188410,-0.490990,0.672139,0.643275,1.252560,24.605499,18.000123,1.335902,1.529872,8.784856
1,1.106377,0.156801,1.273583,1.252009,1.911094,0.883226,11.697757,5.317250,1.667417,1.194068,-0.124440,0.514308,-0.069174,-0.654475,0.643411,0.538587,0.964332,10.548280,12.596606,1.104200,1.306470,1.205778,0.239099,1.367800,1.220141,1.853321,0.961535,8.742572,5.051414,1.719506,1.273306,-0.012767,0.651555,0.083083,-0.566505,0.760875,0.670025,1.013989,9.499813,10.192846,1.091455,1.279133,1.334170,0.383587,1.495090,1.387140,2.049701,1.107185,10.676227,4.929546,1.844064,1.382848,-0.012976,0.787347,0.188410,-0.496130,0.888214,0.802086,1.091451,8.308690,12.427708,1.224738,1.410650,13.681412
2,0.944857,-0.129759,1.405712,1.358765,1.984584,1.222651,6.545482,7.028481,2.487082,1.194068,-0.020367,0.513090,-0.069174,-0.653990,0.740140,0.579497,1.327830,9.631098,6.984980,1.137731,1.342159,1.084284,0.032167,1.492274,1.489302,2.064630,1.338980,7.892284,6.631331,2.483436,1.273306,-0.012688,0.652636,0.083083,-0.565986,0.846851,0.706237,1.347277,17.262883,9.205249,1.209619,1.405501,1.245246,0.178591,1.616746,1.584865,2.188682,1.445457,10.676227,6.431570,2.590225,1.382848,-0.012897,0.788366,0.188410,-0.495569,0.969243,0.836117,1.413670,17.000322,12.427708,1.335902,1.529872,4.891208
3,0.363384,-0.093939,1.570872,1.536692,1.690623,1.284365,14.866269,7.028481,2.487082,1.934928,-0.124440,0.513090,-0.069174,-0.653425,0.256495,0.415856,0.691708,11.924052,15.051692,1.104200,1.306470,0.476812,-0.071300,1.678985,1.657527,1.782884,1.368014,16.369403,6.631331,2.483436,1.950452,-0.011415,0.652636,0.083083,-0.565381,0.416972,0.561390,0.764023,12.087503,16.941423,1.180078,1.373909,0.652421,0.076092,1.799229,1.650774,1.875973,1.501836,19.442996,6.431570,2.590225,2.031057,-0.011628,0.788366,0.188410,-0.494915,0.564102,0.699993,0.849787,12.654506,19.330849,1.308111,1.470261,14.802310
4,0.977161,0.909022,1.372680,1.785790,1.727368,0.821512,14.361144,5.317250,1.667417,1.934928,-0.022596,0.514308,-0.069174,-0.654697,0.320981,0.497677,0.782582,16.509959,14.700965,1.104200,1.306470,1.084284,0.928874,1.398918,1.825753,1.818102,0.932501,13.904425,5.051414,1.719506,1.950452,-0.014994,0.653718,0.083083,-0.566743,0.474289,0.633814,0.847345,15.537756,14.390131,1.150537,1.342317,1.215605,1.032743,1.555918,1.947362,1.945464,1.078995,16.691445,4.929546,1.844064,2.031057,-0.015196,0.789384,0.188410,-0.496387,0.618121,0.768055,0.930342,14.827414,16.586227,1.280320,1.440456,14.389348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-1.542555,-1.383461,-1.534145,-1.523654,-1.432712,-1.523608,-0.425243,-1.527673,-1.611243,-1.769370,-0.124440,-1.948570,-1.287020,-0.654899,-1.936028,-1.970579,-1.852783,-0.457897,-0.380279,-1.947102,-1.941289,-1.284855,-1.174940,-1.277274,-1.269595,-1.210663,-1.274102,-0.395882,-1.268252,-1.336216,-1.435279,-0.120368,-1.534096,-1.082717,-0.566959,-1.531815,-1.550957,-1.485669,-0.419665,-0.423818,-1.537694,-1.532569,-1.096415,-1.017222,-1.090094,-1.084427,-1.042647,-1.091587,-0.356014,-1.078554,-1.140583,-1.209992,-0.120239,-1.271209,-0.971749,-0.496621,-1.272536,-1.285148,-1

In [9]:
#df_merged_fit=pd.DataFrame(df_merged_fit)
#df_merged_fit

In [10]:
X=df_merged_fit.drop(columns=[63])
y=df_merged_fit[63]

In [11]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62
0,0.557208,-0.022299,1.603904,1.714619,1.947839,1.099224,12.285539,7.028481,1.667417,1.194068,-0.019214,0.511873,-0.069174,-0.650032,0.385467,0.347672,1.146081,25.223182,11.719789,1.171262,1.377849,0.719801,0.170122,1.710103,1.825753,2.029412,1.251877,16.420935,6.631331,1.719506,1.273306,-0.120368,0.651555,0.083083,-0.561749,0.531606,0.501037,1.180633,23.732108,15.295428,1.239160,1.437093,0.711703,0.076092,1.829643,1.815545,2.119192,1.389079,19.304116,6.431570,1.844064,1.382848,-0.011707,0.787347,0.18841,-0.490990,0.672139,0.643275,1.252560,24.605499,18.000123,1.335902,1.529872
1,1.106377,0.156801,1.273583,1.252009,1.911094,0.883226,11.697757,5.317250,1.667417,1.194068,-0.124440,0.514308,-0.069174,-0.654475,0.643411,0.538587,0.964332,10.548280,12.596606,1.104200,1.306470,1.205778,0.239099,1.367800,1.220141,1.853321,0.961535,8.742572,5.051414,1.719506,1.273306,-0.012767,0.651555,0.083083,-0.566505,0.760875,0.670025,1.013989,9.499813,10.192846,1.091455,1.279133,1.334170,0.383587,1.495090,1.387140,2.049701,1.107185,10.676227,4.929546,1.844064,1.382848,-0.012976,0.787347,0.18841,-0.496130,0.888214,0.802086,1.091451,8.308690,12.427708,1.224738,1.410650
2,0.944857,-0.129759,1.405712,1.358765,1.984584,1.222651,6.545482,7.028481,2.487082,1.194068,-0.020367,0.513090,-0.069174,-0.653990,0.740140,0.579497,1.327830,9.631098,6.984980,1.137731,1.342159,1.084284,0.032167,1.492274,1.489302,2.064630,1.338980,7.892284,6.631331,2.483436,1.273306,-0.012688,0.652636,0.083083,-0.565986,0.846851,0.706237,1.347277,17.262883,9.205249,1.209619,1.405501,1.245246,0.178591,1.616746,1.584865,2.188682,1.445457,10.676227,6.431570,2.590225,1.382848,-0.012897,0.788366,0.18841,-0.495569,0.969243,0.836117,1.413670,17.000322,12.427708,1.335902,1.529872
3,0.363384,-0.093939,1.570872,1.536692,1.690623,1.284365,14.866269,7.028481,2.487082,1.934928,-0.124440,0.513090,-0.069174,-0.653425,0.256495,0.415856,0.691708,11.924052,15.051692,1.104200,1.306470,0.476812,-0.071300,1.678985,1.657527,1.782884,1.368014,16.369403,6.631331,2.483436,1.950452,-0.011415,0.652636,0.083083,-0.565381,0.416972,0.561390,0.764023,12.087503,16.941423,1.180078,1.373909,0.652421,0.076092,1.799229,1.650774,1.875973,1.501836,19.442996,6.431570,2.590225,2.031057,-0.011628,0.788366,0.18841,-0.494915,0.564102,0.699993,0.849787,12.654506,19.330849,1.308111,1.470261
4,0.977161,0.909022,1.372680,1.785790,1.727368,0.821512,14.361144,5.317250,1.667417,1.934928,-0.022596,0.514308,-0.069174,-0.654697,0.320981,0.497677,0.782582,16.509959,14.700965,1.104200,1.306470,1.084284,0.928874,1.398918,1.825753,1.818102,0.932501,13.904425,5.051414,1.719506,1.950452,-0.014994,0.653718,0.083083,-0.566743,0.474289,0.633814,0.847345,15.537756,14.390131,1.150537,1.342317,1.215605,1.032743,1.555918,1.947362,1.945464,1.078995,16.691445,4.929546,1.844064,2.031057,-0.015196,0.789384,0.18841,-0.496387,0.618121,0.768055,0.930342,14.827414,16.586227,1.280320,1.440456


In [12]:
#X_value=X[['value_eur_21','value_eur_20','value_eur_19']]
#X_value=X_value.dropna(how='all')
#X_value

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,
                                               test_size=0.2,
                                               random_state=True)

In [15]:
X_train.shape

(12000, 63)

In [16]:
y_train.shape

(12000,)

In [17]:
from tensorflow.keras import layers,Sequential
from tensorflow import keras

In [18]:
model=Sequential()
model.add(layers.Dense(32,activation='linear',input_dim=63))
model.add(layers.Dense(16,activation='linear'))
model.add(layers.Dense(1,activation='sigmoid'))
#opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5) #Defining separately to manually modify learning rate
model.compile(loss='mse', optimizer='adam')

In [19]:
#2/24 best one
#model=Sequential()
#model.add(layers.Dense(32,activation='linear',input_dim=63))
#model.add(layers.Dense(16,activation='sigmoid'))
#model.add(layers.Dense(1))
#opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5) #Defining separately to manually modify learning rate
#model.compile(opt, tf.keras.losses.MeanSquaredError(), metrics=['mse','mae'])
#model.compile(loss='mse', optimizer='adam')


In [20]:
#copy from kaggle
#iii = layers.Input(shape=(63,)) #input 
#d1 = layers.Dense(32, activation="sigmoid", input_dim=65)(iii)
#d2 = layers.Dense(16, activation="sigmoid")(d1)
#final = layers.Dense(1, activation="sigmoid")(d2)
#model = keras.Model(inputs=iii, outputs=final)
#opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5) #Defining separately to manually modify learning rate
#model.compile(opt, tf.keras.losses.MeanSquaredError(), metrics=['mse','mae'])


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                2048      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 2,593
Trainable params: 2,593
Non-trainable params: 0
_________________________________________________________________


In [22]:
#opt = tf.keras.optimizers.Adam(learning_rate=1e-3, decay=1e-5) #Defining separately to manually modify learning rate
#model.compile(opt, tf.keras.losses.MeanSquaredError(), metrics=['mse','mae'])

In [23]:
res1=model.fit(X_train,y_train,
               epochs=250,batch_size=10,
               validation_split=0.3)

Epoch 1/250
840/840 [==============================] - 9s 6ms/step - loss: 0.7360 - val_loss: 0.7012
Epoch 2/250
840/840 [==============================] - 5s 6ms/step - loss: 0.7027 - val_loss: 0.6957
Epoch 3/250
840/840 [==============================] - 5s 6ms/step - loss: 0.6994 - val_loss: 0.6944
Epoch 4/250
840/840 [==============================] - 4s 5ms/step - loss: 0.6983 - val_loss: 0.6931
Epoch 5/250
840/840 [==============================] - 3s 3ms/step - loss: 0.6983 - val_loss: 0.6915
Epoch 6/250
840/840 [==============================] - 3s 3ms/step - loss: 0.6969 - val_loss: 0.6909
Epoch 7/250
840/840 [==============================] - 3s 3ms/step - loss: 0.6971 - val_loss: 0.6913
Epoch 8/250
840/840 [==============================] - 3s 3ms/step - loss: 0.6961 - val_loss: 0.6933
Epoch 9/250
840/840 [==============================] - 3s 3ms/step - loss: 0.6958 - val_loss: 0.6916
Epoch 10/250
840/840 [==============================] - 3s 3ms/step - loss: 0.6958 - val_lo

In [24]:
#y_pred=model.predict(X_test)
#pd.DataFrame(y_pred)

In [25]:
#rmse=np.sqrt(mean_squared_error(y_test,y_pred))
#rmse

In [26]:
y_pred1=model.predict(X_test)
rmse=np.sqrt(mean_squared_error(y_test,y_pred1))
rmse

0.8747255215258968